Un cop es tenen els grafs amb la forma que es desitja i en format PyTorch, es pot començar a dissenyar i entrenar la xarxa neuronal que donarà una solució al problema.

Els grafs (en format .pt) es troben a les carpetes "train_tsp_pt" i "test_tsp_pt", per a l'entrenament i per al test corresponentment.

# Importacions

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from pathlib import Path

from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv

In [57]:
# Carpetes
train_path = Path("Datasets/train_pt")
test_path = Path("Datasets/test_pt")

# Carregar tots els grafs de train i test
graphs_train = [torch.load(f) for f in sorted(train_path.glob("*.pt"))]
graphs_test = [torch.load(f) for f in sorted(test_path.glob("*.pt"))]

print(f"Nombre de grafs de train: {len(graphs_train)}")
print(f"Nombre de grafs de test: {len(graphs_test)}")

Nombre de grafs de train: 1514
Nombre de grafs de test: 4


In [58]:
# Exemples
data = graphs_train[300]
print(data)
print(data.x)

Data(x=[27, 2], edge_index=[2, 351], edge_attr=[351, 1], y=[27], id=[27, 1])
tensor([[1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 1.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])


In [107]:
# Normalitzar valors arestes
#for g in graphs_train + graphs_test:
#    mean, std = g.edge_attr.mean(), g.edge_attr.std()
#    g.edge_attr = (g.edge_attr - mean) / (std + 1e-8)

In [59]:
class TSPGraphDataset(Dataset):
    def __init__(self, graphs):
        self.graphs = graphs

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]

train_dataset = TSPGraphDataset(graphs_train)
test_dataset = TSPGraphDataset(graphs_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Disseny GNN

In [60]:
class TSPGNN(nn.Module):
    def __init__(self, hidden_channels=64):
        """
        GNN senzilla per predir el següent node en el TSP.
        Node features = [initial, current]
        """
        super(TSPGNN, self).__init__()
        self.conv1 = GCNConv(2, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.conv5 = GCNConv(hidden_channels, hidden_channels)
        self.conv6 = GCNConv(hidden_channels, hidden_channels)
        self.out = nn.Linear(hidden_channels, 1)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x = F.relu(self.conv1(x, edge_index, edge_weight=edge_attr.view(-1)))
        x = F.relu(self.conv2(x, edge_index, edge_weight=edge_attr.view(-1)))
        x = F.relu(self.conv3(x, edge_index, edge_weight=edge_attr.view(-1)))
        x = F.relu(self.conv4(x, edge_index, edge_weight=edge_attr.view(-1)))
        x = F.relu(self.conv5(x, edge_index, edge_weight=edge_attr.view(-1)))
        x = F.relu(self.conv6(x, edge_index, edge_weight=edge_attr.view(-1)))

        logits = self.out(x).squeeze(-1)  # [num_nodes]

        return logits

# Training

In [ ]:
# Nota pel 13/11. fer un batch enorme, nomes s'esta entrenant un graf per epoch. normalitzar les distancies. repensar el loss.

## Configuració

In [61]:
# --- CONFIGURACIÓ ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TSPGNN(hidden_channels=64).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

## Funcions

In [62]:
# --- FUNCIÓ D'ENTRENAMENT --- batch>1
def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()

        logits = model(data)  # [num_total_nodes]
        batch = data.batch    # indica a quin graf pertany cada node

        # Convertim logits i targets per graf:
        # agrupem els nodes per graf dins del batch
        loss = 0.0
        num_graphs = batch.max().item() + 1

        for g in range(num_graphs):
            mask = (batch == g)
            graph_logits = logits[mask].unsqueeze(0)  # [1, num_nodes_g]
            target_index = data.y[mask].argmax().unsqueeze(0)  # [1]
            loss += criterion(graph_logits, target_index)

        loss = loss / num_graphs  # mitjana de pèrdues per graf
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)

In [63]:
# --- FUNCIÓ D'AVALUACIÓ --- batch>1
@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    total_correct_nodes = 0
    total_nodes = 0
    total_correct_top1 = 0
    total_graphs = 0
    normalized_ranks = []

    for data in loader:
        data = data.to(device)
        logits = model(data).view(-1)
        batch = data.batch
        num_graphs = batch.max().item() + 1

        for g in range(num_graphs):
            mask = (batch == g)
            probs = F.softmax(logits[mask], dim=0)

            # --- NodeAcc ---
            pred = torch.zeros_like(probs)
            pred[probs.argmax()] = 1.0
            total_correct_nodes += (pred == data.y[mask]).sum().item()
            total_nodes += mask.sum().item()

            # --- Top1Acc ---
            correct_node_index = data.y[mask].argmax()
            predicted_node_index = probs.argmax()
            if predicted_node_index == correct_node_index:
                total_correct_top1 += 1
            total_graphs += 1

            # --- Normalized Rank ---
            sorted_indices = torch.argsort(probs, descending=True)
            rank = (sorted_indices == correct_node_index).nonzero(as_tuple=True)[0].item() + 1
            normalized_rank = (rank - 1) / (mask.sum().item() - 1)
            normalized_ranks.append(normalized_rank)

    node_acc = total_correct_nodes / total_nodes
    top1_acc = total_correct_top1 / total_graphs
    mean_normalized_rank = sum(normalized_ranks) / len(normalized_ranks)

    return node_acc, top1_acc, mean_normalized_rank

### Batch=1

In [26]:
# --- FUNCIÓ D'ENTRENAMENT --- batch=1
def train(model, loader, optimizer, criterion, device):
    """
    Entrena el model durant una època sobre els grafs del loader.
    """
    model.train()
    total_loss = 0.0

    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()

        # Forward pass
        logits = model(data)             # [num_nodes]
        logits = logits.unsqueeze(0)     # [1, num_nodes]

        # Target = índex del node correcte
        target = data.y.argmax().unsqueeze(0)  # [1]

        # Cross entropy loss
        loss = criterion(logits, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

In [27]:
# --- FUNCIÓ D'AVALUACIÓ --- batch=1
@torch.no_grad()
def evaluate(model, loader, device):
    """
    Avalua el rendiment del model amb 3 mètriques:
    - node_acc: precisió global per node (0 i 1)
    - top1_acc: proporció de grafs on s'encerta el node correcte
    - mean_rank: posició mitjana normalitzada del node correcte dins del rànquing
    """
    model.eval()
    total_correct_nodes = 0
    total_nodes = 0
    total_correct_top1 = 0
    total_graphs = 0
    normalized_ranks = []

    for data in loader:
        data = data.to(device)
        logits = model(data).view(-1)

        # Softmax per convertir a probabilitats sobre tots els nodes del graf
        probs = F.softmax(logits, dim=0)

        # --- NodeAcc ---
        pred = torch.zeros_like(probs)
        pred[probs.argmax()] = 1.0
        total_correct_nodes += (pred == data.y).sum().item()
        total_nodes += data.num_nodes

        # --- Top1Acc ---
        correct_node_index = data.y.argmax()
        predicted_node_index = probs.argmax()
        if predicted_node_index == correct_node_index:
            total_correct_top1 += 1
        total_graphs += 1

        # --- Normalized Rank ---
        sorted_indices = torch.argsort(probs, descending=True)
        rank = (sorted_indices == correct_node_index).nonzero(as_tuple=True)[0].item() + 1
        normalized_rank = (rank - 1) / (data.num_nodes - 1)
        normalized_ranks.append(normalized_rank)

    node_acc = total_correct_nodes / total_nodes
    top1_acc = total_correct_top1 / total_graphs
    mean_normalized_rank = sum(normalized_ranks) / len(normalized_ranks)

    return node_acc, top1_acc, mean_normalized_rank

## Entrenament

In [64]:
# --- ENTRENAMENT ---
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    loss = train(model, train_loader, optimizer, criterion, device)
    node_acc, top1_acc, mean_rank = evaluate(model, train_loader, device)
    print(f"Epoch {epoch:02d} | Loss: {loss:.3f} | NodeAcc: {node_acc:.3f} | Top1Acc: {top1_acc:.3f} | MeanRank: {mean_rank:.2f}")

Epoch 01 | Loss: 3.943 | NodeAcc: 0.976 | Top1Acc: 0.069 | MeanRank: 0.36
Epoch 02 | Loss: 3.895 | NodeAcc: 0.976 | Top1Acc: 0.071 | MeanRank: 0.35
Epoch 03 | Loss: 3.881 | NodeAcc: 0.976 | Top1Acc: 0.065 | MeanRank: 0.35
Epoch 04 | Loss: 3.878 | NodeAcc: 0.976 | Top1Acc: 0.061 | MeanRank: 0.38
Epoch 05 | Loss: 3.877 | NodeAcc: 0.976 | Top1Acc: 0.065 | MeanRank: 0.36
Epoch 06 | Loss: 3.886 | NodeAcc: 0.976 | Top1Acc: 0.067 | MeanRank: 0.35
Epoch 07 | Loss: 3.873 | NodeAcc: 0.976 | Top1Acc: 0.067 | MeanRank: 0.34
Epoch 08 | Loss: 3.861 | NodeAcc: 0.976 | Top1Acc: 0.065 | MeanRank: 0.35
Epoch 09 | Loss: 3.855 | NodeAcc: 0.976 | Top1Acc: 0.066 | MeanRank: 0.37
Epoch 10 | Loss: 3.851 | NodeAcc: 0.976 | Top1Acc: 0.068 | MeanRank: 0.35


# Test

In [34]:
model.eval()
print(graphs_test[0])
print(graphs_test[0].edge_index)
print(graphs_test[0].weight)
with torch.no_grad():
    data = graphs_test[0]
    logits = model(data)
    pred_node = torch.argmax(logits).item()
    print(f"Node predit com a target: {pred_node}")

Data(edge_index=[2, 142845], initial=[535], current=[535], target=[535], weight=[142845])
tensor([[  0,   0,   0,  ..., 532, 532, 533],
        [  1,   2,   3,  ..., 533, 534, 534]])
tensor([2264, 3788, 3713,  ...,  753, 7518, 7980])
Node predit com a target: 496


In [29]:
def tsp_inference(model, pyg_graph, device="cpu"):
    """
    Donat un graf de test, utilitza el model per predir el tour complet.
    Retorna l'ordre de nodes visitats i la distància total.
    """
    model.eval()
    G = pyg_graph.clone()  # clonem per no modificar l'original
    G = G.to(device)
    G.weight = G.weight.float()

    # Identificar nodes inicials
    node_ids = list(range(G.initial.size(0)))
    initial_idx = torch.where(G.initial == 1)[0].item()
    current_idx = torch.where(G.current == 1)[0].item()

    visited = [initial_idx]
    total_distance = 0.0

    while len(node_ids) > 3:  # fins que quedin 3 nodes (initial, current, un altre)
        # Predir target
        with torch.no_grad():
            logits = model(G)
            probs = torch.sigmoid(logits)
            probs[visited] = -1  # ignorem els nodes ja visitats

        next_idx = torch.argmax(probs).item()

        # Calcular la distància entre current i next
        mask = ((G.edge_index[0] == current_idx) & (G.edge_index[1] == next_idx)) | \
               ((G.edge_index[0] == next_idx) & (G.edge_index[1] == current_idx))
        edge_weight = G.weight[mask]
        if edge_weight.numel() > 0:
            total_distance += edge_weight.mean().item()  # o .item() si només n’hi ha una

        # Actualitzar els atributs del graf
        G.current[:] = 0
        G.current[next_idx] = 1
        G.target[:] = 0  # netegem target

        # Eliminar el node anterior (ja visitat, excepte l’inicial)
        if current_idx != initial_idx:
            mask = torch.ones_like(G.initial, dtype=torch.bool)
            mask[current_idx] = False
            G.initial = G.initial[mask]
            G.current = G.current[mask]
            G.target = G.target[mask]

            keep_edges = (mask[G.edge_index[0]] & mask[G.edge_index[1]])
            G.edge_index = G.edge_index[:, keep_edges]
            G.weight = G.weight[keep_edges]

        # Afegim a la llista de visitats i actualitzem
        visited.append(next_idx)
        current_idx = next_idx
        node_ids = list(range(G.initial.size(0)))

    # Tancar el cicle tornant a l’inicial
    mask = ((G.edge_index[0] == current_idx) & (G.edge_index[1] == initial_idx)) | \
           ((G.edge_index[0] == initial_idx) & (G.edge_index[1] == current_idx))
    edge_weight = G.weight[mask]
    if edge_weight.numel() > 0:
        total_distance += edge_weight.mean().item()

    visited.append(initial_idx)
    return visited, total_distance


In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for i, graph in enumerate(graphs_test):
    path, length = tsp_inference(model, graph, device=device)
    print(f"Graf {i}: camí = {path}, longitud total = {length:.2f}")

RuntimeError: index 534 is out of bounds for dimension 0 with size 534